[Professor GitHub Repository](https://github.com/bardhprenkaj/ML_labs/blob/main/resources/project/project_description.pdf)

[Project GitHub Repository](https://github.com/Naxetee/ML_FinalProject)

# Index

- **Python Libraries**
- **Task 1**
    - *Reading Datasets*
    - *Exercise 1*
    - *Exercise 2*
    - *Exercise 3*
    - *Exercise 4*
    - *Exercise 5*
    - *Writing Clean Datasets*


# Python Libraries


In [114]:
import pandas as pd
import numpy as np
import datetime as dt

# Task 1

During this first task we will apply some **pre-processing** operations to our raw data. So we will directly modify the data in *data/dataset/*, so that the data in *data/dataset_fixed/* will always remain unmodified.

## Reading the Datasets

At first, let's read the **.csv** files using a *pandas* function called **read_csv** whose parameters are:
- *filepath*: the path of the .csv files.
- *header*: will be always 0 to skip the columns' name row.
- *names*: gives a name to each column of the table.


In [115]:
fixed_data_path  = "../../data/dataset_fixed/sample"

ap_fixed   = pd.read_csv(fixed_data_path+'/anagraficapazientiattivi.csv',      header=0 ,names=['idcentro','idana','sesso','annodiagnosidiabete','tipodiabete','scolarita','statocivile','professione','origine','annonascita','annoprimoaccesso','annodecesso'])
diag_fixed = pd.read_csv(fixed_data_path+'/diagnosi.csv',                      header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
elp_fixed  = pd.read_csv(fixed_data_path+'/esamilaboratorioparametri.csv',     header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
ei_fixed   = pd.read_csv(fixed_data_path+'/esamistrumentali.csv',              header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pdf_fixed  = pd.read_csv(fixed_data_path+'/prescrizionidiabetefarmaci.csv',    header=0 ,names=['idcentro','idana','data','codiceatc','quantita','idpasto','descrizionefarmaco'])
pdnf_fixed = pd.read_csv(fixed_data_path+'/prescrizionidiabetenonfarmaci.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pnd_fixed  = pd.read_csv(fixed_data_path+'/prescrizioninondiabete.csv',        header=0 ,names=['idcentro','idana','data','codiceamd','valore'])

In [116]:
read_data_path  = "../../data/dataset/sample"

ap   = pd.read_csv(read_data_path+'/anagraficapazientiattivi.csv',      header=0 ,names=['idcentro','idana','sesso','annodiagnosidiabete','tipodiabete','scolarita','statocivile','professione','origine','annonascita','annoprimoaccesso','annodecesso'])
diag = pd.read_csv(read_data_path+'/diagnosi.csv',                      header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
elp  = pd.read_csv(read_data_path+'/esamilaboratorioparametri.csv',     header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
ei   = pd.read_csv(read_data_path+'/esamistrumentali.csv',              header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pdf  = pd.read_csv(read_data_path+'/prescrizionidiabetefarmaci.csv',    header=0 ,names=['idcentro','idana','data','codiceatc','quantita','idpasto','descrizionefarmaco'])
pdnf = pd.read_csv(read_data_path+'/prescrizionidiabetenonfarmaci.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pnd  = pd.read_csv(read_data_path+'/prescrizioninondiabete.csv',        header=0 ,names=['idcentro','idana','data','codiceamd','valore'])

In [117]:
# Display example observations
print(ap.shape)
ap.head()

(250000, 12)


,idcentro,idana,sesso,annodiagnosidiabete,tipodiabete,scolarita,statocivile,professione,origine,annonascita,annoprimoaccesso,annodecesso
0,89,13370,M,2007.0,5,NaN,NaN,NaN,NaN,1946,NaN,NaN
1,100,3080,M,1988.0,5,NaN,NaN,NaN,NaN,1936,1991.0,NaN
2,72,15730,M,2015.0,5,NaN,NaN,NaN,NaN,1965,2015.0,NaN
3,150,3799,F,1986.0,5,NaN,NaN,NaN,NaN,1928,NaN,NaN
4,141,4432,F,2010.0,5,NaN,NaN,NaN,NaN,1949,2011.0,NaN


In [118]:
print(diag.shape)
diag.head()

(4427337, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,1980-01-01,AMD247,491.20
1,1,5,1986-01-01,AMD247,272.0
2,1,5,1987-01-01,AMD083,NaN
3,1,5,1987-01-01,AMD247,401
4,1,5,1997-12-01,AMD247,410


In [119]:
print(elp.shape)
elp.head()

(28628530, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2005-01-18,AMD001,169.0
1,1,5,2005-01-18,AMD002,76.0
2,1,5,2005-01-18,AMD004,135.0
3,1,5,2005-01-18,AMD005,80.0
4,1,5,2005-06-06,AMD007,169.0


In [120]:
print(ei.shape)
ei.head()

(1015740, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2006-01-04,AMD051,N
1,1,5,2006-11-14,AMD041,P
2,1,5,2006-11-20,AMD040,P
3,1,5,2007-06-01,AMD040,N
4,1,5,2008-01-16,AMD040,N


In [121]:
print(pdf.shape)
pdf.head()

(7012648, 7)


,idcentro,idana,data,codiceatc,quantita,idpasto,descrizionefarmaco
0,82,782,2008-04-02,A10BA02,1.0,1,GLUCOPHAGE*60CPR RIV 1000MG
1,82,782,2008-04-02,A10BA02,1.0,3,GLUCOPHAGE*60CPR RIV 1000MG
2,82,782,2008-04-02,A10BA02,1.0,5,GLUCOPHAGE*60CPR RIV 1000MG
3,82,782,2008-04-02,A10BB09,3.0,1,DIAMICRON*60CPR R.M. 30MG
4,82,782,2008-05-02,A10BA02,1.0,1,GLUCOPHAGE*60CPR RIV 1000MG


In [122]:
print(pdnf.shape)
pdnf.head()

(548467, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2008-06-20,AMD152,NaN
1,1,5,2013-08-27,AMD152,NaN
2,1,5,2013-12-31,AMD086,S
3,1,5,2013-12-31,AMD228,S
4,1,6,2005-06-08,AMD086,S


In [123]:
print(pnd.shape)
pnd.head()

(5083861, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2005-01-18,AMD121,C09AA05
1,1,5,2005-01-18,AMD124,C10AA05
2,1,5,2005-01-18,AMD124,C10AX06
3,1,5,2005-06-21,AMD121,C09AA05
4,1,5,2005-06-21,AMD124,C10AA05


## Exercise 1

**Select Events of Interest -**
We want only patients with at leats one cardiovascular event in their trajectories. These events have the folloging codes:
- **AMD047**: Myocardial infarction
- **AMD048**: Coronary angioplasty
- **AMD049**: Coronary bypass
- **AMD071**: Ictus
- **AMD081**: Lower limb angioplasty
- **AMD082**: Peripheral By-pass Lower Limbs
- **AMD208**: Revascularization of intracranial and neck vessels
- **AMD303**: Ischemic stroke

Now let's try to filter all the tables containing the parameter **codiceamd** so as to select just the rows describing one cardio-vascular event.


In [124]:
codes = ['AMD047', 'AMD048', 'AMD049', 'AMD071', 'AMD081', 'AMD082', 'AMD208', 'AMD303']

def selectEvents(df:pd.DataFrame, events_codes:list) -> pd.DataFrame:
    return df[df.codiceamd.isin(events_codes)]

## pnd_cardEvents = selectEvents(pnd,codes)

## pdnf_cardEvents = selectEvents(pdnf,codes)

## ei_cardEvents = selectEvents(ei,codes)

## elp_cardEvents = selectEvents(elp,codes)

diag_cardEvents = selectEvents(diag,codes)
print("BEFORE pre-processing: ", diag.shape)
print("AFTER  pre-processing: ", diag_cardEvents.shape)
diag_cardEvents.head()


BEFORE pre-processing:  (4427337, 5)
AFTER  pre-processing:  (233204, 5)


,idcentro,idana,data,codiceamd,valore
7,1,5,2000-12-02,AMD049,S
9,1,5,2001-03-22,AMD049,S
13,1,5,2002-02-11,AMD049,S
15,1,5,2002-02-14,AMD049,S
17,1,5,2003-01-17,AMD049,S


We realise that the only table with these cardio-vascular events is **diagnosi**, so we create a new DataFrame with all the rows that has one of these events, and we call it: **diag_cardEvents**. And from this table, we extract the id's of the patients that have suffered at least one cardio-vascular event during his life:

In [125]:
patientsOfInterest = diag_cardEvents.filter(['idcentro','idana']).drop_duplicates()
print("Number of patients with at least one cardio-vascular event: ", patientsOfInterest.shape[0], "/", ap.shape[0] , "patients.")

Number of patients with at least one cardio-vascular event:  50000 / 250000 patients.


Finally, we have to delete from the rest of the tables all the information which is not associated to the remaining patients:

In [126]:
ap   = pd.merge(ap, patientsOfInterest,   on=['idcentro','idana'], how='right')[ap_fixed.columns]
diag = pd.merge(diag, patientsOfInterest, on=['idcentro','idana'], how='right')[diag_fixed.columns]
elp  = pd.merge(elp, patientsOfInterest,  on=['idcentro','idana'], how='right')[elp_fixed.columns]
ei   = pd.merge(ei, patientsOfInterest,   on=['idcentro','idana'], how='right')[ei_fixed.columns]
pdf  = pd.merge(pdf, patientsOfInterest,  on=['idcentro','idana'], how='right')[pdf_fixed.columns]
pdnf = pd.merge(pdnf, patientsOfInterest, on=['idcentro','idana'], how='right')[pdnf_fixed.columns]
pnd  = pd.merge(pnd, patientsOfInterest,  on=['idcentro','idana'], how='right')[pnd_fixed.columns]

In [127]:
print(f"We still have: {ap.shape[0]}/{ap_fixed.shape[0]} patients")

We still have: 50000/250000 patients


## Exercise 2

**Invalid feature cleaning** - check for dates and time intervals $[x, y]$ such that $y < x$ and not $x ≤ y$.
Check for years that do not make sense (e.g., events before the birth of a particular patient).


In [128]:
# We see the number of patients that we have before checking that their dates are valid
ap.shape

(50000, 12)

In [129]:
# First we check that the patient's birth year, the year of the diabetes diagnosis and the year of thier first access is not NaN
# Then, we check that the birth year, the year of first access and the year of their diabetes diagnosis are all smaller than their death year 

ap = ap[(ap['annonascita'].notna()) & (ap['annoprimoaccesso'].notna()) & (ap['annodiagnosidiabete'].notna()) & (ap['annonascita']<=ap['annodecesso']) & (ap['annoprimoaccesso']<=ap['annodecesso']) & (ap['annodiagnosidiabete']<=ap['annodecesso'])]
print(ap.shape)

# Then we add the patients that have NaN as the death year because in that case some of the previous conditions are evaluated as false but 
# we assume taht if the death year is NaN then the patient is still alive so the years would fulfill the conditions

ap = pd.concat([ap, ap[ap['annodecesso'].isna()]])
print(ap.shape)

# Lastly, we check that the patient's birth year is smaller than the year of the diabetes diagnosis and is also smaller than the year of 
# first access

ap = ap[(ap['annonascita']<=ap['annodiagnosidiabete']) & (ap['annonascita']<=ap['annoprimoaccesso'])]
print(ap.shape)

(4722, 12)
(4722, 12)
(4722, 12)


In [130]:
# We then see the new number of patients that we have and check if we have made an error and if there are duplicates in the table

ap_test = ap.filter(items=['idcentro', 'idana'])
print(ap_test.shape)
ap_test.drop_duplicates()
print(ap_test.shape)

(4722, 2)
(4722, 2)


In [131]:
# For the rest of the tables, we do the same checks

def exercise2(df:pd.DataFrame, test:pd.DataFrame) -> pd.DataFrame:

    # First we merge the table to obtain only the data of the patients that have valid dates in the ap table
    df = df.merge(test, on=['idcentro', 'idana'])
    df = df[df['data'].notna()]

    # Then, we assure that the dates of the table we are purging are valid (not NaN and not a day that doesn't exist in the calendar) and that the year is first on the dates
    df['data'] = pd.to_datetime(df['data'], yearfirst=True)

    # Now we create a new table with the columns that have dates from the ap table and the table that we are purging
    ap_filter = ap.filter(items=['idcentro', 'idana', 'annonascita', 'annoprimoaccesso', 'annodecesso'])
    df = df.merge(ap_filter, on=['idcentro', 'idana'])

    # Now we take the rows of this table that have valid dates, this means that we have to check:
    # That all the tests performed on a patient have a smaller date than the death year of the patient
    df = df[df['data'].dt.strftime("%Y").astype(float) <= df['annodecesso']]

    # Then, as we did with the ap table, we add the rows where the death year of the patient is NaN (because we assume that is still alive and therefore it fulfills the conditions)
    df = pd.concat([df, ap_filter[ap_filter['annodecesso'].isna()]])

    # Lastly, we check that the test were performed after the year of first access and after the year of birth of the patient
    df = df[(df['data'].dt.strftime("%Y").astype(float) >= df['annoprimoaccesso']) & (df['data'].dt.strftime("%Y").astype(float) >= df['annonascita'])]

    # In the pdf table in particular we need to check that the prescriptions were made after the year of diabetes diagnosis because the medicines prescripted are for people
    # with diabetes
    if 'annodiagnosidiabete' in df.columns:
        df = df[(df['data'].dt.strftime("%Y").astype(float) >= df['annodiagnosidiabete'])]

    # Now, we take only the columns that were in the purged table to begin with
    return df.filter(items=df.columns)

We apply the previous function to the rest of our tables

In [132]:
diag = exercise2(diag,ap_test)
print(f"Diag: {diag_fixed.shape[0]} => {diag.shape[0]}")

Diag: 4427337 => 190618


In [133]:
elp = exercise2(elp,ap_test)
print(f"Elp: {elp_fixed.shape[0]} => {elp.shape[0]}")

Elp: 28628530 => 665848


In [134]:
ei = exercise2(ei,ap_test)
print(f"Ei: {ei_fixed.shape[0]} => {ei.shape[0]}")

Ei: 1015740 => 24196


In [135]:
pdf = exercise2(pdf,ap_test)
print(f"Pdf: {pdf_fixed.shape[0]} => {pdf.shape[0]}")

Pdf: 7012648 => 189497


In [136]:
pdnf = exercise2(pdnf,ap_test)
print(f"Pdnf: {pdnf_fixed.shape[0]} => {pdnf.shape[0]}")

Pdnf: 548467 => 16714


In [137]:
pnd = exercise2(pnd,ap_test)
print(f"Pnd: {pnd_fixed.shape[0]} => {pnd.shape[0]}")

Pnd: 5083861 => 192695


In [138]:
print(f"We still have: {ap.shape[0]}/{ap_fixed.shape[0]} patients")

We still have: 4722/250000 patients


## Exercise 3

First of all, we convert the data column of each dataframe into datetime type

In [139]:
pnd['data']  = pd.to_datetime(pnd['data'],  format="%Y-%m-%d")
diag['data'] = pd.to_datetime(diag['data'], format="%Y-%m-%d")
elp['data']  = pd.to_datetime(elp['data'],  format="%Y-%m-%d")
pdf['data']  = pd.to_datetime(pdf['data'],  format="%Y-%m-%d")
pdnf['data'] = pd.to_datetime(pdnf['data'], format="%Y-%m-%d")
ei['data']   = pd.to_datetime(ei['data'],   format="%Y-%m-%d")

In [140]:
def exercise3(df: pd.DataFrame) -> pd.DataFrame:
    maxDates = df.groupby(['idcentro','idana']).agg({'data' : ['max']})
    minDates = df.groupby(['idcentro','idana']).agg({'data' : ['min']})
    return maxDates, minDates

We calculate maximum and minimum dates for each table and for every single patient (idcentro, idana)

In [141]:
pndMax,  pndMin  = exercise3(pnd)
diagMax, diagMin = exercise3(diag)
elpMax,  elpMin  = exercise3(elp)
pdfMax,  pdfMin  = exercise3(pdf)
pdnfMax, pdnfMin = exercise3(pdnf)
eiMax,   eiMin   = exercise3(ei)

Now we merge all the maximum dates of all dataframes, and we take **maxim** as the maximum date of all tables for each patient

In [142]:
maxim = pd.merge(pndMax,diagMax,on=['idcentro','idana'])
maxim = pd.merge(maxim,elpMax,on=['idcentro','idana'])
max2  = pd.merge(pdnfMax,pdfMax,on=['idcentro','idana'])
max2  = pd.merge(eiMax,max2,on=['idcentro','idana'])
maxim = pd.merge(maxim,max2,on=['idcentro','idana'])
maxim = maxim.max(axis=1)
maxim.head()

idcentro  idana
1         5       2013-12-31
          38      2015-12-18
          65      2017-11-09
          74      2012-12-06
          90      2012-09-25
dtype: datetime64[ns]

Same with the minimum date

In [143]:
minim = pd.merge(pndMin,diagMin,on=['idcentro','idana'])
minim = pd.merge(minim,elpMin,on=['idcentro','idana'])
min2  = pd.merge(pdnfMin,pdfMin,on=['idcentro','idana'])
min2  = pd.merge(eiMin,min2,on=['idcentro','idana'])
minim = pd.merge(minim,min2,on=['idcentro','idana'])
minim = minim.min(axis=1)
minim.head()

idcentro  idana
1         5       1997-12-01
          38      1998-01-01
          65      2000-06-08
          74      2000-01-08
          90      2002-01-01
dtype: datetime64[ns]

Now we compute the difference between the maximum dates and the minimum, this is the trajectory, and we compare it with a month (30 days)

In [144]:
dif = maxim-minim
print("DIFFERENCE TABLE before comparing:",dif.shape[0],"patients.\n",dif.head())

month = pd.Timedelta("30 days")
dif = dif[dif>month]
print("\nDIFFERENCE TABLE after comparing:",dif.shape[0],"patients.\n",dif.head())

print(f"\nSo we have to maintain {dif.shape[0]}/{ap.shape[0]} patients")

DIFFERENCE TABLE before comparing: 2194 patients.
 idcentro  idana
1         5       5874 days
          38      6560 days
          65      6363 days
          74      4716 days
          90      3920 days
dtype: timedelta64[ns]

DIFFERENCE TABLE after comparing: 2179 patients.
 idcentro  idana
1         5       5874 days
          38      6560 days
          65      6363 days
          74      4716 days
          90      3920 days
dtype: timedelta64[ns]

So we have to maintain 2179/4722 patients


Now we convert the **dif** dataframe so as to be available for the drop operation on every table

In [145]:
dif = pd.DataFrame(dif.index.to_list(), columns=['idcentro','idana'])

In [146]:
ap   = ap.merge(dif, on=['idcentro','idana'])
diag = diag.merge(dif, on=['idcentro','idana'])
elp  = elp.merge(dif, on=['idcentro','idana'])
ei   = ei.merge(dif, on=['idcentro','idana'])
pdf  = pdf.merge(dif, on=['idcentro','idana'])
pdnf = pdnf.merge(dif, on=['idcentro','idana'])
pnd  = pnd.merge(dif, on=['idcentro','idana'])
ap.head()

,idcentro,idana,sesso,annodiagnosidiabete,tipodiabete,scolarita,statocivile,professione,origine,annonascita,annoprimoaccesso,annodecesso
0,1,5,M,1986.0,5,2.0,2.0,9.0,NaN,1942,1990.0,2014.0
1,1,38,F,1978.0,5,2.0,3.0,9.0,NaN,1924,1998.0,2015.0
2,1,65,M,1997.0,5,2.0,1.0,9.0,NaN,1942,1997.0,2018.0
3,1,74,M,1995.0,5,2.0,2.0,9.0,NaN,1930,1996.0,2013.0
4,1,90,M,2002.0,5,3.0,2.0,9.0,NaN,1943,2002.0,2013.0


In [147]:
print(f"We still have: {ap.shape[0]}/{ap_fixed.shape[0]} patients")

We still have: 2179/250000 patients


## Exercise 4

**Modify the actual ranges of** ***esamilaboratorioparameteri*** **-** see this Table:

| Code | Descriptive Name | True Range |
|----------|----------|----------|
| AMD004 | Systolic blood pressure | 40 <= x <= 200 |
| AMD005 | Diastolic blood pressure | 40 <= x <= 130 |
| AMD007 | Fasting blood glucose | 50 <= x <= 500 |
| AMD008 | HbAlc | 5 <= x <= 15 |
| AMD009 | Creatininemia | Not Available |
| AMD111 | Microalbuminuria | Not Available |
| STITCH001 | BMI | Not Available |
| STITCH002 | LDL Choresterlo | 30 <= x <= 30|
| STITCH003 | Non-HDL Cholesterlo | 60 <= x <= 330 |
| STITCH004 | eGFR MDRD | Not Available |
| STITCH005 | eGFR CKD-EPI | Not Available |

What we are doing is to use a **MinMaxScaler** from *sklearn* library so as to make the values described in the upper table fit into the true ranges. For the **NAN** ranges, we do nothing.

In [148]:
labels = ['AMD004', 'AMD005', 'AMD007', 'AMD008', 'AMD009', 'AMD111', 'STITCH001', 'STITCH002', 'STITCH003', 'STITCH004', 'STITCH005']

print("BEFORE RE-SCALING:\n")
print("    AMD    Min  /  Max")
print("========================")

for label in labels:
    aux = elp[elp.codiceamd == label].filter(['valore']).values.tolist()
    if len(aux) > 0:
        print(f"{label}:    {np.min(aux):.1f} / {np.max(aux):.1f}")
    else:
        print(f"{label}:  is null")

BEFORE RE-SCALING:

    AMD    Min  /  Max
AMD004:    0.0 / 430.0
AMD005:    0.0 / 750.0
AMD007:    0.0 / 862.0
AMD008:    0.0 / 98.0
AMD009:    nan / nan
AMD111:    0.0 / 8647.0
STITCH001:  is null
STITCH002:  is null
STITCH003:  is null
STITCH004:  is null
STITCH005:  is null


In [149]:
from sklearn.preprocessing import MinMaxScaler

NAN = np.nan
newRanges = [['AMD004', 'AMD005', 'AMD007', 'AMD008', 'AMD009', 'AMD111', 'STITCH001', 'STITCH002', 'STITCH003', 'STITCH004', 'STITCH005'],
            [40.0,40.0,50.0,5.0,NAN,NAN,NAN,30.0,60.0,NAN,NAN],
            [200.0,130.0,500.0,15.0,NAN,NAN,NAN,300.0,330.0,NAN,NAN]]

def scaling(df:pd.DataFrame, label:str, minValue:float, maxValue:float) -> pd.DataFrame:
    if (minValue != NAN) and (maxValue != NAN) and (df[df.codiceamd == label].shape[0]>0):
        aux = df[df.codiceamd == label].copy()
        min_max_scaler = MinMaxScaler(feature_range=(minValue,maxValue))
        aux[['valore']] = min_max_scaler.fit_transform(aux[['valore']])
        df[df.codiceamd == label] = aux
    return df


# Let's apply the function to a new DataFrame copied from elp_new
for k in range(len(newRanges[0])):
    elp = scaling(elp, newRanges[0][k], newRanges[1][k], newRanges[2][k])

Now we ensure manually that it has worked by printing the maximum and minimum values for each AMD Code

In [150]:
print("\n\nAFTER RE-SCALING:\n")
print("    AMD    Min  /  Max")
print("========================")

for label in labels:
    aux = elp[elp.codiceamd == label].filter(['valore']).values.tolist()
    if len(aux) > 0:
        print(f"{label}:    {np.min(aux):.1f} / {np.max(aux):.1f}")
    else:
        print(f"{label}:  is null")




AFTER RE-SCALING:

    AMD    Min  /  Max
AMD004:    40.0 / 200.0
AMD005:    40.0 / 130.0
AMD007:    50.0 / 500.0
AMD008:    5.0 / 15.0
AMD009:    nan / nan
AMD111:    nan / nan
STITCH001:  is null
STITCH002:  is null
STITCH003:  is null
STITCH004:  is null
STITCH005:  is null


So it matches with the table given by the exercise

## Exercise 5

**Cohort selection and label definition** - use only those patients that after all the previous steps contain at least two events before calculating the label. Let $\mathcal{P}=\lbrace p_1, \cdots,p_n\rbrace$ be the set of all patients in the dataset. Let $d(e^i_k)$ be the date of the last event $e_k$ for patient $p_i\in\mathcal{P}$. The label of the patient $p_i$ is calculated as follows:

$y(p_i)=\begin{dcases}
                1 & \text{if, within }d(e^i_k)-6 \text{ months, }p_i\text{ has a cardiovascular event}\\
                0 & \text{otherwise}
        \end{dcases}$

Eliminate the patients that have a trajectory shorten or equal to 6 months.


We first create a DataFrame that contains all the patients and their cardiovascular events, each one with its date (it's already computed as **diag_cardEvents**):

In [151]:
diag_cardEvents.head()

,idcentro,idana,data,codiceamd,valore
7,1,5,2000-12-02,AMD049,S
9,1,5,2001-03-22,AMD049,S
13,1,5,2002-02-11,AMD049,S
15,1,5,2002-02-14,AMD049,S
17,1,5,2003-01-17,AMD049,S


Now for each patient of the frame **diag_cardEvents**, we are going to take its two last cardiovascular events dates, so as to check if they happened with less than 6 months of difference. For this, we define the function **labeling**

In [152]:
def exercise5(df:pd.DataFrame) -> pd.DataFrame:

    # First we define a dataframe that contains the ids and the dates of every cardiovascular event, sorted by descending date
    cardEvents_sorted = df.sort_values('data',ascending=False)

    # Now we create a new dataset called classes, that represents the result of the function y(p_i) defined previously:
    classes = pd.DataFrame(columns=['idcentro','idana','class'])

    # Now for each patient (idcentro,idana), we check if it verify the condition explained previously:
    for label, subDataframe in cardEvents_sorted.groupby(['idcentro','idana']):

        dates = subDataframe['data'].dropna().values

        # We ignore those patients who have just one event
        if len(dates) > 1:

            # We extract the first and the 2 lastest dates and convert them into datetime objects
            minDate = dates[-1]
            maxDates = dates[0:2]
            minDate = dt.date.fromisoformat(minDate)
            maxDates = dt.date.fromisoformat(maxDates[0]), dt.date.fromisoformat(maxDates[1])

            # We ignore patients with a trajectory shorter than 6 months (6*30 days)
            if maxDates[0]-minDate > dt.timedelta(days=6*30):

                # We calculate the difference between the latest 2 dates and compare it with 6 months
                c = maxDates[0]-maxDates[1] < dt.timedelta(days=6*30)

                # We add a new column with the label and the class of the patient
                classes.loc[classes.count()[0]] = [label[0], label[1], int(c)]

    return classes

Now, in the **classes** dataframe, we have labeled the patients that have more than one event and a trajectory larger than 6 months

In [153]:
labels = exercise5(diag_cardEvents)

So we do an inner merge operation again with every dataframe to take just the patients of the frame **classes**

In [158]:
ap   = pd.merge(ap,   labels, on=['idcentro','idana'])[ap_fixed.columns.tolist()+['class']]
diag = pd.merge(diag, labels, on=['idcentro','idana'])[diag_fixed.columns]
elp  = pd.merge(elp,  labels, on=['idcentro','idana'])[elp_fixed.columns]
ei   = pd.merge(ei,   labels, on=['idcentro','idana'])[ei_fixed.columns]
pdf  = pd.merge(pdf,  labels, on=['idcentro','idana'])[pdf_fixed.columns]
pdnf = pd.merge(pdnf, labels, on=['idcentro','idana'])[pdnf_fixed.columns]
pnd  = pd.merge(pnd,  labels, on=['idcentro','idana'])[pnd_fixed.columns]
patientsOfInterest_labeled = pd.merge(patientsOfInterest, labels, on=['idcentro','idana'])


In [159]:
print(f"We still have: {ap.shape}/{ap_fixed.shape} patients")

We still have: (1555, 13)/(250000, 12) patients


## Writing Clean Datasets

In [161]:
write_data_path = "../../data/dataset_preprocessed"

ap.to_csv(write_data_path+"/anagraficapazientiattivi.csv",        index=False)
diag.to_csv(write_data_path+"/diagnosi.csv",                      index=False)
elp.to_csv(write_data_path+"/esamilaboratorioparametri.csv",      index=False)
ei.to_csv(write_data_path+"/esamistrumentali.csv",                index=False)
pdf.to_csv(write_data_path+"/prescrizionidiabetefarmaci.csv",     index=False)
pdnf.to_csv(write_data_path+"/prescrizionidiabetenonfarmaci.csv", index=False)
pnd.to_csv(write_data_path+"/prescrizioninondiabete.csv",         index=False)